# CNN - Transfer Learning

Test VGG16 vs EfficientNet

## Define directories

In [34]:
import os

In [4]:
project_path='/home/remo/code/victor-ocn/covid19-project/covid19/Covid19/raw_data/cloud/train_test/'
!tree -d {project_path}

/home/remo/code/victor-ocn/covid19-project/covid19/Covid19/raw_data/cloud/train_test/
├── test
│   ├── COVID19
│   ├── NORMAL
│   ├── OPACITY
│   └── PNEUMONIA
└── train
    ├── COVID19
    ├── NORMAL
    ├── OPACITY
    └── PNEUMONIA

10 directories


In [5]:
train_dir = os.path.join(project_path, 'train')
test_dir = os.path.join(project_path, 'test')

## Create Generators

https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

In [11]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_generator = image_dataset_from_directory(train_dir,
                                               labels="inferred",
                                               color_mode='grayscale',
                                               batch_size = 32,
                                               image_size=(224,224),
                                               seed=42,
                                               label_mode = "categorical")

test_generator = image_dataset_from_directory(test_dir,
                                              labels="inferred",
                                              batch_size = 32,
                                              color_mode='grayscale',
                                              image_size=(224,224),
                                              seed=42,
                                              label_mode = "categorical")

Found 20227 files belonging to 4 classes.
Found 5059 files belonging to 4 classes.


## Test a Simple CNN model

In [30]:
#from tensorflow.keras.layers import Rescaling
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(224,224,1)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(4))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(4))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(4))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))

    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [31]:
model_homemade = load_own_model()
model_homemade.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 224, 224, 1)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 215, 215, 16)      1616      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 53, 53, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 46, 46, 32)        32800     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 11, 11, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 6, 6, 32)         

In [33]:
es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = model_homemade.fit(train_generator,
                             batch_size = 16,
                             epochs = 100,
                             callbacks=[es])

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
evaluate_cnn = model_homemade.evaluate(test_generator)
print(evaluate_cnn)

In [35]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)